# AutoGen அடிப்படை மாதிரி

இந்த குறியீட்டு மாதிரியில், [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework-ஐ பயன்படுத்தி ஒரு அடிப்படை முகவரியை உருவாக்குவீர்கள்.

இந்த மாதிரியின் நோக்கம், பின்னர் வேறு முகவரியின் வடிவங்களை செயல்படுத்தும் போது பயன்படுத்தப்படும் படிகளை உங்களுக்குக் காட்டுவது.


## தேவையான பைதான் தொகுப்புகளை இறக்குமதி செய்யவும்


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## கிளையன்ட் உருவாக்கம்

இந்த உதாரணத்தில், LLM-ஐ அணுக [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) பயன்படுத்தப்படும்.

`model` என்பது `gpt-4o-mini` என வரையறுக்கப்பட்டுள்ளது. GitHub Models சந்தையில் கிடைக்கும் மற்றொரு மாடலுக்கு மாறி, வெவ்வேறு முடிவுகளைப் பார்க்க முயற்சிக்கவும்.

ஒரு விரைவான சோதனையாக, நாம் ஒரு எளிய கேள்வியை இயக்கப் போகிறோம் - `பிரான்சின் தலைநகரம் என்ன`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ஏஜென்டை வரையறுப்பது

இப்போது `client` அமைக்கப்பட்டு, அது செயல்படுகிறது என்பதை உறுதிப்படுத்திய பிறகு, `AssistantAgent` உருவாக்குவோம். ஒவ்வொரு ஏஜென்டுக்கும் கீழ்கண்டவைகளை ஒதுக்கலாம்:  
**name** - பல ஏஜென்ட்களின் செயல்பாடுகளில் இதை குறிப்பிடுவதற்கு உதவும் சுருக்கமான பெயர்.  
**model_client** - நீங்கள் முந்தைய படியில் உருவாக்கிய client.  
**tools** - ஏஜென்டு ஒரு பணியை முடிக்க பயன்படுத்தக்கூடிய கருவிகள்.  
**system_message** - LLM-இன் பணியை, நடத்தை மற்றும் குரலை வரையறுக்கும் metaprompt.  

LLM எப்படி பதிலளிக்கிறது என்பதைப் பார்க்க system message-ஐ மாற்றலாம். `tools` பற்றி நான்காவது பாடத்தில் விரிவாகக் காண்போம்.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## ஏஜென்டை இயக்கவும்

கீழே உள்ள செயல்பாடு ஏஜென்டை இயக்கும். புதிய செய்தியுடன் ஏஜென்டின் நிலையை புதுப்பிக்க `on_message` முறைமையை பயன்படுத்துகிறோம்.

இந்தக் கட்டத்தில், பயனரிடமிருந்து புதிய செய்தியுடன் நிலையை புதுப்பிக்கிறோம், அது `"Plan me a great sunny vacation"`.

LLM எப்படி வேறுபடுகிறதென்று பார்க்க, செய்தி உள்ளடக்கத்தை மாற்றலாம்.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. எங்கள் தரச்சிறப்பிற்காக முயற்சி செய்தாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
